# Correlating Traces and Logs using Python

## Import OpenTelemetry Modules for Traces and Logs

In [ ]:
# Import Trace dependencies...
from opentelemetry import baggage, trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import (
    ConsoleSpanExporter,
    BatchSpanProcessor
)
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.trace import Status, StatusCode
import logging, datetime, random, socket, sys, time, uuid

# Import Log dependencies...
from opentelemetry._logs import set_logger_provider
from opentelemetry.exporter.otlp.proto.grpc._log_exporter import (
    OTLPLogExporter,
)
from opentelemetry.sdk._logs.export import (
    ConsoleLogExporter,
    BatchLogRecordProcessor,
    SimpleLogRecordProcessor
)
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler

otlp_endpoint = "localhost:4317"
print(otlp_endpoint)

env = "otel-adventure"
print(env)

## Create a **Tracer**

See Section 4 for details.

In [ ]:
tracerProvider = TracerProvider(resource=Resource.create({
    "service.name": __name__,
    "service.instance.id": str(uuid.uuid4()),
    "deployment.environment.name": env,
    "host.name": socket.gethostname(),
}))
print(tracerProvider.__dict__)

tracerProvider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint=otlp_endpoint, insecure=True)))
tracer = trace.get_tracer("python", tracer_provider=tracerProvider)
print(tracer.resource.__dict__)

## Create a **Logger**

See Section 5 for details.

In [ ]:
loggerProvider = LoggerProvider(resource=Resource.create({
    "service.name": __name__,
    "service.instance.id": str(uuid.uuid4()),
    "deployment.environment.name": "otel-adventure",
    "host.name": socket.gethostname(),
}))
print(loggerProvider.__dict__)

log_exporter = OTLPLogExporter(endpoint=otlp_endpoint, insecure=True)

# FOR LAB EXERCISES: No batching of log events...
loggerProvider.add_log_record_processor(SimpleLogRecordProcessor(log_exporter))

logger = logging.getLogger()
logger.handlers.clear()
logger.addHandler(logging.StreamHandler(sys.stderr))

# In Python, use setLevel() to enable DEBUG output for the logger instance
logger.setLevel(logging.DEBUG)

# Attach OTLP handler to root logger
handler = LoggingHandler(logger_provider=loggerProvider)
logger.addHandler(handler)

## Send trace spans and logs it to the collector


In [ ]:
def getTracer(service_name):
    tracerProvider = TracerProvider(resource=Resource.create({
        "service.name": service_name,
        "service.instance.id": str(uuid.uuid4()),
        "deployment.environment.name": env,
        "host.name": socket.gethostname(),
    }))
    tracerProvider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint=otlp_endpoint, insecure=True)))
    # print(tracerProvider._resource.__dict__)
    return trace.get_tracer("python", tracer_provider=tracerProvider)
    
def frontend():
    frontend_tracer = getTracer("frontend")
    with frontend_tracer.start_as_current_span("frontend_process") as frontend_span:
        logging.info("Processing web transaction...") 
        time.sleep(random.random())
        handle_checkout()
        time.sleep(random.random())
    
        frontend_span.set_status(Status(StatusCode.OK))
        logging.info("Transaction complete.")
        trace_id = trace.format_trace_id(frontend_span.context.trace_id)
        print(f"Trace ID: {trace_id}")
        print(f"View trace in Datadog: https://app.datadoghq.com/apm/traces?query=%40otel.trace_id%3A{trace_id}")
        print(f"View flow map: https://app.datadoghq.com/apm/traces?query_a=%40otel.trace_id%3A{trace_id}&view=traces&traceQuery=a")
        return trace_id

def handle_checkout():
    checkout_tracer = getTracer("checkout")
    with checkout_tracer.start_as_current_span("checkout_process") as checkout_span:
        logging.info("Handling checkout...")
        checkout_span.set_attribute("order_num", int(datetime.datetime.timestamp(datetime.datetime.now())*1000) % 100000)
        time.sleep(random.random())
        handle_payment()
        time.sleep(random.random())
        handle_shipping()
        time.sleep(random.random())
    
        checkout_span.set_status(Status(StatusCode.OK))
        logging.info("Checkout complete.")
        
def handle_payment():
    payment_tracer = getTracer("payment")
    with payment_tracer.start_as_current_span("payment_process") as payment_span:
        logging.info("Handling payment...")
        payment_span.set_attribute("payment_id", str(uuid.uuid4()))
        if (random.random() < 0.1):
            payment_span.set_status(Status(StatusCode.ERROR))
        else:
            time.sleep(random.random())
            payment_span.set_status(Status(StatusCode.OK))
        logging.info("Payment complete.")
    
def handle_shipping():
    shipping_tracer = getTracer("shipping")
    with shipping_tracer.start_as_current_span("shipping_process") as shipping_span:
        logging.info("Handling shipping...")
        shipping_span.set_attribute("tracking_num", str(uuid.uuid4()))
        time.sleep(random.random())
        logging.info("Shipping complete.")

trace_id = frontend()    

### Updated flowmap with each service represented separately
<img src="imgs/log-correlation.png" width="600"/>

#### End of Section